In [1]:
import torch
import time
from tt_utils import *
from fcn_approx_utils import GMM, BGMM

In [2]:
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dim_list = [2,5,10]
mix_list = [5,10,20]
s_list =[0.15,0.3,0.45]

Ndim = len(dim_list)
Nmix = len(mix_list)
Ns = len(s_list)
Nt = 5

tt_data_err = torch.empty((Ndim, Nmix, Ns,Nt))
tt_data_time = torch.empty((Ndim, Nmix, Ns,Nt))
tt_data_rank = torch.empty((Ndim, Nmix, Ns,Nt))

bgmm_data_err = torch.empty((Ndim, Nmix, Ns,Nt))
bgmm_data_time = torch.empty((Ndim, Nmix, Ns,Nt))
L = 1.0

for i, dim_ in enumerate(dim_list):
    for j, nmix_ in enumerate(mix_list):
        for k, s_ in enumerate(s_list):
            for p in range(Nt):
                print("###########")
                print(i,j,k,p)
                print(dim_,nmix_,s_,p)
                print("###########")
                gmm = GMM(n=dim_,nmix=nmix_,L=L,mx_coef=None,mu=None,s=s_, device=device)
                pdf = gmm.pdf
                ndata_train = int(1e4)*dim_
                x_train = 2*L*(-0.5 + torch.rand((ndata_train,dim_)).to(device))
                y_train = pdf(x_train).view(-1,1)
                ndata_test = 10000
                x_test = 2*L*(-0.5 + torch.rand((ndata_test,dim_)).to(device))
                y_test = pdf(x_test)
                x_test_np = x_test.detach().cpu().numpy()
                y_test_np = y_test.detach().cpu().numpy()
                bgmm = BGMM(nmix = nmix_)
                x_train_np = gmm.generate_sample(ndata_train)
                bgmm.load_data(x_train_np)
                t01 = time.time()
                bgmm.fit()
                t02=time.time()
                y_bgmm = bgmm.pdf(x_test_np)
                mse_bgmm =  (((y_bgmm.reshape(-1)-y_test_np.reshape(-1))/(1e-9+np.abs(y_test_np).reshape(-1)))**2).mean()
                 
                dt = t02-t01
                bgmm_data_err[i,j,k,p] = 1*mse_bgmm
                bgmm_data_time[i,j,k,p] = dt
                print("mse_bgmm: ", mse_bgmm)
                
                n_discretization = torch.tensor([200]*dim_).to(device)
                domain = [torch.linspace(-L,L,n_discretization[i_]).to(device) for i_ in range(dim_)] 
                t1 = time.time()
                tt = cross_approximate(fcn=pdf,  max_batch=10**6, domain=domain, 
                                        rmax=200, nswp=20, eps=1e-3, verbose=False, 
                                        kickrank=10, device=device)
                t2 = time.time()

                y_tt =  get_value(tt_model=tt, x=x_test.to(device),  domain=domain, 
                                    n_discretization=n_discretization , max_batch=10**5, device=device)

                mse_tt = (((y_tt.view(-1)-y_test.view(-1))/(1e-9+y_test.view(-1).abs()))**2).mean()
                print("mse_tt: ", mse_tt)
                tt_data_err[i,j,k,p] = 1*mse_tt
                tt_data_rank[i,j,k,p] = max(tt.ranks_tt)
                tt_data_time[i,j,k,p] = (t2-t1)


                torch.save({'dim_list':dim_list,'s_list':s_list,'mix_list':mix_list,
                          'tt_data_err':tt_data_err,
                         'tt_data_time':tt_data_time,
                         'tt_data_rank':tt_data_rank},'tt_data_2.pt')
                torch.save({'dim_list':dim_list,'s_list':s_list,'mix_list':mix_list,
                          'bgmm_data_err':bgmm_data_err,
                         'bgmm_data_time':bgmm_data_time},'bgmm_data_2.pt')

###########
0 0 0 0
2 5 0.15 0
###########


/idiap/temp/sshetty/miniconda/envs/pyml/lib/python3.9/site-packages/sklearn/mixture/_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


mse_bgmm:  tensor(0.3203)
mse_tt:  tensor(3.3938e-05)
###########
0 0 0 1
2 5 0.15 1
###########


/idiap/temp/sshetty/miniconda/envs/pyml/lib/python3.9/site-packages/sklearn/mixture/_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


mse_bgmm:  tensor(0.1323)
mse_tt:  tensor(2.8331e-05)
###########
0 0 0 2
2 5 0.15 2
###########


...

mse_bgmm:  tensor(9.1601)
mse_tt:  tensor(1.8098e-06)
###########
1 0 1 2
5 5 0.3 2
###########
mse_bgmm:  tensor(0.0185)
mse_tt:  tensor(1.7394e-06)
###########
1 0 1 3
5 5 0.3 3
###########
mse_bgmm:  tensor(0.0244)
mse_tt:  tensor(1.9602e-06)
###########
1 0 1 4
5 5 0.3 4
###########


/idiap/temp/sshetty/miniconda/envs/pyml/lib/python3.9/site-packages/sklearn/mixture/_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


mse_bgmm:  tensor(0.8076)
mse_tt:  tensor(2.0173e-06)
###########
1 0 2 0
5 5 0.45 0
###########


/idiap/temp/sshetty/miniconda/envs/pyml/lib/python3.9/site-packages/sklearn/mixture/_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


mse_bgmm:  tensor(0.0026)


In [ ]:
tt_data = torch.load('tt_data_2.pt')
bgmm_data = torch.load('bgmm_data_2.pt')

In [ ]:
tt_mean = tt_data['tt_data_err'].mean(dim=(-1,-2))
tt_std = tt_data['tt_data_err'].std(dim=(-1,-2))

bgmm_mean = bgmm_data['bgmm_data_err'].mean(dim=(-1,-2))
bgmm_std = bgmm_data['bgmm_data_err'].std(dim=(-1,-2))

In [ ]:
tt_bgmm = (tt_data['tt_data_err']/bgmm_data['bgmm_data_err']).detach().cpu()

In [ ]:
y_mean = tt_bgmm.mean(dim=(-1,-2))
y_std = tt_bgmm.std(dim=(-1,-2))

In [ ]:
dim_list = tt_data['dim_list']
dim_list

In [ ]:
mix_list = tt_data['mix_list']
mix_list

In [ ]:
s_list = tt_data['s_list']
s_list

In [ ]:
y_mean.shape

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 10})

# Define your data
conditions = ['k=5','k=10', 'k=20']
m1_means =  y_mean[0] # Mean values for Method 1
m1_stdevs = y_std[0]   # Standard deviations for Method 1
m2_means = y_mean[1]   # Mean values for Method 2
m2_stdevs =  y_std[1]   # Standard deviations for Method 2
m3_means = y_mean[2]   # Mean values for Method 2
m3_stdevs =  y_std[2]   # Standard deviations for Method 2
m4_means = y_mean[3]   # Mean values for Method 2
m4_stdevs = y_std[3]   # Standard deviations for Method 2



# Set the width of the bars
bar_width = 0.1

# Set the positions of the bars on the x-axis
r1 = np.arange(len(conditions))-0.1
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Plot the bars
plt.bar(r1, m1_means,  width=bar_width, label='d=10', capsize=5)
plt.bar(r2, m2_means,  width=bar_width, label='d=20', capsize=5)
plt.bar(r3, m3_means,  width=bar_width, label='d=30', capsize=5)
plt.bar(r4, m4_means,  width=bar_width, label='d=40', capsize=5)

# Add labels, title, and legend
# plt.xlabel('number of mixture components',fontsize='12')
plt.ylabel('Error ratio TT/NN',fontsize='13')
# plt.title('TT vs NN')
plt.xticks([r + bar_width/2 for r in range(len(conditions))], conditions, fontsize=12)
plt.legend(ncol=4)
plt.yticks(fontsize=12)
plt.yscale('log')
plt.savefig('tt_vs_nn.jpeg', bbox_inches='tight',pad_inches=0.01, dpi=1000)
# Show the plot
plt.show()
